<a href="https://colab.research.google.com/github/hinapyon/Sampling-Frequency-Estimating-of-Human-Activity-Acceleration-Data-using-Transformer-based-Regression/blob/main/my_data%2Bhasc(Transformer-based-Regression-Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
from decimal import Decimal #小数点桁落ちをなくすためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score #機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
#定数を定義
BINS = 50  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #機械学習のトレーニングデータの割合
N_ESTIMATORS = 100  #決定木の数

In [62]:
#ディレクトリ内のAMWS020のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [63]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [64]:
#自分の加速度データのCSVファイルから3軸加速度を取得する関数
#なんか分割したら3, 4, 5色々おかしくなった
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[3]))
            AccY.append(float(row[4]))
            AccZ.append(float(row[5]))

    return AccX, AccY, AccZ

In [65]:
#自分の加速度データのCSVファイルから3軸加速度を取得する関数
def get_accelerationHasc(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[1]))
            AccY.append(float(row[2]))
            AccZ.append(float(row[3]))

    return AccX, AccY, AccZ

In [66]:
#静止区間を除去する関数
def acc_to_remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i = 0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [67]:
#連続する2サンプルの差分を取る関数
def calculate_differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    #DifferenceAcc = [math.fabs(ResultantAcc[i + 1] * 100000 - ResultantAcc[i] * 100000) for i in range(len(ResultantAcc) - 1)]  #100000倍して誤差を取る
    DifferenceAcc = [math.fabs(Decimal(ResultantAcc[i + 1]) - Decimal(ResultantAcc[i])) for i in range(len(ResultantAcc) - 1)]  #連続する2サンプルの誤差を取る
    return DifferenceAcc

In [68]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が各加速度の差分データの最小値〜最大値）
def create_histogram2(DifferenceAcc_list: list[float]) -> np.histogram:
    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    for i in range(len(DifferenceAcc_list)):
        min_value = min(DifferenceAcc_list[i])
        max_value = max(DifferenceAcc_list[i])
        #DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, density=True) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [69]:
#各加速度データをダウンサンプリングする関数
def resampling_Acc(originHz: int, newHz: int, AccX: list[float], AccY: list[float], AccZ: list[float], Hz: np.array) -> tuple[list[float], list[float], list[float], np.array]:
    i = 0   #カウンター変数

    while (Hz[i] == originHz):
        originlen = len(AccX[i])    #元々のデータの長さ
        sampling_factor = int(originlen * (newHz/originHz)) #ダウンサンプリングした後のデータの長さ
        newAccX = signal.resample(AccX[i], sampling_factor)    #データをダウンサンプリング
        newAccY = signal.resample(AccY[i], sampling_factor)    #データをダウンサンプリング
        newAccZ = signal.resample(AccZ[i], sampling_factor)    #データをダウンサンプリング
        AccX.append(newAccX)   #ダウンサンプリングデータを加速度データに追加
        AccY.append(newAccY)   #ダウンサンプリングデータを加速度データに追加
        AccZ.append(newAccZ)   #ダウンサンプリングデータを加速度データに追加
        Hz = np.append(Hz, newHz)   #ダウンサンプリングレートを追加
        i += 1

    return AccX, AccY, AccZ, Hz

In [70]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [106]:
path = "/content/drive/MyDrive/my_walk_data(100Hz15minutesTo20seconds)/"  #Google colab用
#path = "my_walk_data(100Hz15minutesTo20seconds)/"  #local用

In [107]:
#一連の流れ
Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_Hz_and_filename(Hz_and_filename)
Hz = np.array(Hz)

#使う変数を宣言
readAccX, readAccY, readAccZ = [], [], []   #データ読み込む用
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み二次元配列に格納
for i in filename:
    readAccX, readAccY, readAccZ = get_acceleration(path+i)
    AccX.append(readAccX), AccY.append(readAccY), AccZ.append(readAccZ)

#各加速度データをダウンサンプリング
for i in range(9, 1, -1):
    AccX, AccY, AccZ, Hz = resampling_Acc(100, i * 10, AccX, AccY, AccZ, Hz)

#静止区間を除去
for i in range(len(Hz)):
    ResultantAcc.append(acc_to_remove_stationary_intervals(AccX[i], AccY[i], AccZ[i]))

In [108]:
#ヒストグラム作成
for i in range(len(ResultantAcc)):
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc[i]))
DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

In [109]:
pathHasc = "/content/drive/MyDrive/hasc/"
filename = os.listdir(pathHasc) #引数のパスのディレクトリの中のファイル名一覧を取得
#filename.remove(".DS_Store")

In [110]:
#使う変数を宣言
readAccX, readAccY, readAccZ = [], [], []   #データ読み込む用
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み二次元配列に格納
for i in filename:
    readAccX, readAccY, readAccZ = get_accelerationHasc(pathHasc+i)
    AccX.append(readAccX), AccY.append(readAccY), AccZ.append(readAccZ)

HzHasc = np.ones(len(filename)) * 100

#各加速度データをダウンサンプリング
for i in range(9, 1, -1):
    AccX, AccY, AccZ, HzHasc = resampling_Acc(100, i * 10, AccX, AccY, AccZ, HzHasc)

#静止区間を除去
for i in range(len(HzHasc)):
    ResultantAcc.append(acc_to_remove_stationary_intervals(AccX[i], AccY[i], AccZ[i]))

In [111]:
#ヒストグラム作成
for i in range(len(ResultantAcc)):
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc[i]))
DifferenceAcc_histHasc = create_histogram2(DifferenceAcc_list)

In [112]:
pathKaggle = "/content/drive/MyDrive/11.Walk/"
filename = os.listdir(pathKaggle) #引数のパスのディレクトリの中のファイル名一覧を取得
#filename.remove(".DS_Store")

In [113]:
#使う変数を宣言
readAccX, readAccY, readAccZ = [], [], []   #データ読み込む用
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み二次元配列に格納
for i in filename:
    readAccX, readAccY, readAccZ = get_accelerationHasc(pathKaggle+i)
    AccX.append(readAccX), AccY.append(readAccY), AccZ.append(readAccZ)

HzKaggle = np.ones(len(filename)) * 100

#各加速度データをダウンサンプリング
for i in range(9, 1, -1):
    AccX, AccY, AccZ, HzKaggle = resampling_Acc(100, i * 10, AccX, AccY, AccZ, HzKaggle)

#静止区間を除去
for i in range(len(HzKaggle)):
    ResultantAcc.append(acc_to_remove_stationary_intervals(AccX[i], AccY[i], AccZ[i]))

In [114]:
#ヒストグラム作成
for i in range(len(ResultantAcc)):
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc[i]))
DifferenceAcc_histKaggle = create_histogram2(DifferenceAcc_list)

In [115]:
#標準化
scaler = StandardScaler()
# Fit the scaler to x_train
scaler.fit(DifferenceAcc_hist)
# Use the scaler to transform x_train and x_test
DifferenceAcc_hist = scaler.transform(DifferenceAcc_hist)

In [116]:
#標準化
scaler = StandardScaler()
# Fit the scaler to x_train
scaler.fit(DifferenceAcc_histHasc)
# Use the scaler to transform x_train and x_test
DifferenceAcc_histHasc = scaler.transform(DifferenceAcc_histHasc)

In [117]:
#標準化
scaler = StandardScaler()
# Fit the scaler to x_train
scaler.fit(DifferenceAcc_histKaggle)
# Use the scaler to transform x_train and x_test
DifferenceAcc_histKaggle = scaler.transform(DifferenceAcc_histKaggle)

In [118]:
#transformer regression
#各種パラメータ
NUM_HEADS = 2
KEY_DIM = 25
BINS = 50
DROPOUT = 0.1
N = 1

# Define the input shape
input_shape = (BINS,)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
inputs_encoder = layers.Input(shape=input_shape)
inputs_decoder = layers.Input(shape=output_shape)

In [119]:
#Encoderに対する入力の形状
x_encoder = layers.Reshape((1, 50))(inputs_encoder)

In [120]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(BINS * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(BINS, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [121]:
x = layers.Flatten()(x_encoder)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="relu")(x)

In [122]:
model = keras.Model(inputs=inputs_encoder, outputs=outputs)

In [123]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.mean_squared_error,
    metrics=[keras.metrics.mean_squared_error],
)

In [144]:
mae = []
mse = []
r2 = []

In [145]:
EPOCHS = 150

In [146]:
#自分ので構築→Hascので試す
history = model.fit(DifferenceAcc_hist, Hz, batch_size=32, epochs=EPOCHS, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histHasc), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzHasc))
mse.append(mean_squared_error(y_pred, HzHasc))
r2.append(r2_score(y_pred, HzHasc))

Epoch 1/150
12/12 [==============================] - 0s 14ms/step - loss: 206.1710 - mean_squared_error: 206.1710 - val_loss: 181.4717 - val_mean_squared_error: 181.4717
Epoch 2/150
12/12 [==============================] - 0s 10ms/step - loss: 118.7249 - mean_squared_error: 118.7249 - val_loss: 187.9138 - val_mean_squared_error: 187.9138
Epoch 3/150
12/12 [==============================] - 0s 9ms/step - loss: 97.8471 - mean_squared_error: 97.8471 - val_loss: 207.3856 - val_mean_squared_error: 207.3856
Epoch 4/150
12/12 [==============================] - 0s 14ms/step - loss: 72.2696 - mean_squared_error: 72.2696 - val_loss: 189.1618 - val_mean_squared_error: 189.1618
Epoch 5/150
12/12 [==============================] - 0s 14ms/step - loss: 71.4100 - mean_squared_error: 71.4100 - val_loss: 161.5666 - val_mean_squared_error: 161.5666
Epoch 6/150
12/12 [==============================] - 0s 14ms/step - loss: 62.1394 - mean_squared_error: 62.1394 - val_loss: 154.8667 - val_mean_squared_error

In [147]:
#自分ので構築→Kaggleので試す
history = model.fit(DifferenceAcc_hist, Hz, batch_size=32, epochs=EPOCHS, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histKaggle), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzKaggle))
mse.append(mean_squared_error(y_pred, HzKaggle))
r2.append(r2_score(y_pred, HzKaggle))

Epoch 1/150
12/12 [==============================] - 0s 14ms/step - loss: 53.5683 - mean_squared_error: 53.5683 - val_loss: 156.2957 - val_mean_squared_error: 156.2957
Epoch 2/150
12/12 [==============================] - 0s 9ms/step - loss: 49.7851 - mean_squared_error: 49.7851 - val_loss: 125.8118 - val_mean_squared_error: 125.8118
Epoch 3/150
12/12 [==============================] - 0s 9ms/step - loss: 59.6372 - mean_squared_error: 59.6372 - val_loss: 135.9685 - val_mean_squared_error: 135.9685
Epoch 4/150
12/12 [==============================] - 0s 10ms/step - loss: 48.7236 - mean_squared_error: 48.7236 - val_loss: 147.9486 - val_mean_squared_error: 147.9486
Epoch 5/150
12/12 [==============================] - 0s 9ms/step - loss: 51.6817 - mean_squared_error: 51.6817 - val_loss: 132.8299 - val_mean_squared_error: 132.8299
Epoch 6/150
12/12 [==============================] - 0s 9ms/step - loss: 56.8133 - mean_squared_error: 56.8133 - val_loss: 123.1384 - val_mean_squared_error: 123.1

In [148]:
#Hascで構築→自分ので試す
history = model.fit(DifferenceAcc_histHasc, HzHasc, batch_size=32, epochs=EPOCHS, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_hist), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, Hz))
mse.append(mean_squared_error(y_pred, Hz))
r2.append(r2_score(y_pred, Hz))

Epoch 1/150
12/12 [==============================] - 0s 12ms/step - loss: 219.5724 - mean_squared_error: 219.5724 - val_loss: 505.4459 - val_mean_squared_error: 505.4459
Epoch 2/150
12/12 [==============================] - 0s 10ms/step - loss: 135.2576 - mean_squared_error: 135.2576 - val_loss: 403.7343 - val_mean_squared_error: 403.7342
Epoch 3/150
12/12 [==============================] - 0s 9ms/step - loss: 123.9668 - mean_squared_error: 123.9668 - val_loss: 368.7053 - val_mean_squared_error: 368.7053
Epoch 4/150
12/12 [==============================] - 0s 9ms/step - loss: 102.6973 - mean_squared_error: 102.6973 - val_loss: 329.4151 - val_mean_squared_error: 329.4151
Epoch 5/150
12/12 [==============================] - 0s 9ms/step - loss: 78.6258 - mean_squared_error: 78.6258 - val_loss: 281.6004 - val_mean_squared_error: 281.6004
Epoch 6/150
12/12 [==============================] - 0s 9ms/step - loss: 71.4794 - mean_squared_error: 71.4794 - val_loss: 260.6092 - val_mean_squared_erro

In [149]:
#Hascで構築→Kaggleので試す
history = model.fit(DifferenceAcc_histHasc, HzHasc, batch_size=32, epochs=EPOCHS, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histKaggle), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzKaggle))
mse.append(mean_squared_error(y_pred, HzKaggle))
r2.append(r2_score(y_pred, HzKaggle))

Epoch 1/150
12/12 [==============================] - 0s 12ms/step - loss: 48.5272 - mean_squared_error: 48.5272 - val_loss: 341.0424 - val_mean_squared_error: 341.0424
Epoch 2/150
12/12 [==============================] - 0s 10ms/step - loss: 47.6753 - mean_squared_error: 47.6753 - val_loss: 332.7953 - val_mean_squared_error: 332.7953
Epoch 3/150
12/12 [==============================] - 0s 10ms/step - loss: 47.6088 - mean_squared_error: 47.6088 - val_loss: 378.8881 - val_mean_squared_error: 378.8881
Epoch 4/150
12/12 [==============================] - 0s 10ms/step - loss: 54.9006 - mean_squared_error: 54.9006 - val_loss: 332.0627 - val_mean_squared_error: 332.0627
Epoch 5/150
12/12 [==============================] - 0s 11ms/step - loss: 49.1213 - mean_squared_error: 49.1213 - val_loss: 339.0618 - val_mean_squared_error: 339.0618
Epoch 6/150
12/12 [==============================] - 0s 10ms/step - loss: 56.1502 - mean_squared_error: 56.1502 - val_loss: 364.1015 - val_mean_squared_error: 3

In [150]:
#Kaggleで構築→自分ので試す
history = model.fit(DifferenceAcc_histKaggle, HzKaggle, batch_size=32, epochs=EPOCHS, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_hist), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, Hz))
mse.append(mean_squared_error(y_pred, Hz))
r2.append(r2_score(y_pred, Hz))

Epoch 1/150
43/43 [==============================] - 0s 9ms/step - loss: 147.8425 - mean_squared_error: 147.8425 - val_loss: 402.7919 - val_mean_squared_error: 402.7919
Epoch 2/150
43/43 [==============================] - 0s 8ms/step - loss: 94.5662 - mean_squared_error: 94.5662 - val_loss: 365.2750 - val_mean_squared_error: 365.2750
Epoch 3/150
43/43 [==============================] - 0s 8ms/step - loss: 73.9235 - mean_squared_error: 73.9235 - val_loss: 391.7784 - val_mean_squared_error: 391.7784
Epoch 4/150
43/43 [==============================] - 0s 8ms/step - loss: 68.9172 - mean_squared_error: 68.9172 - val_loss: 343.6192 - val_mean_squared_error: 343.6192
Epoch 5/150
43/43 [==============================] - 0s 8ms/step - loss: 67.3234 - mean_squared_error: 67.3234 - val_loss: 371.3581 - val_mean_squared_error: 371.3581
Epoch 6/150
43/43 [==============================] - 0s 7ms/step - loss: 63.4647 - mean_squared_error: 63.4647 - val_loss: 377.1284 - val_mean_squared_error: 377.1

In [151]:
#Kaggleので構築→HASCので試す
history = model.fit(DifferenceAcc_histKaggle, HzKaggle, batch_size=32, epochs=EPOCHS, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histHasc), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzHasc))
mse.append(mean_squared_error(y_pred, HzHasc))
r2.append(r2_score(y_pred, HzHasc))

Epoch 1/150
43/43 [==============================] - 1s 13ms/step - loss: 42.9852 - mean_squared_error: 42.9852 - val_loss: 377.6434 - val_mean_squared_error: 377.6434
Epoch 2/150
43/43 [==============================] - 1s 12ms/step - loss: 44.9129 - mean_squared_error: 44.9129 - val_loss: 331.1776 - val_mean_squared_error: 331.1776
Epoch 3/150
43/43 [==============================] - 1s 12ms/step - loss: 49.9693 - mean_squared_error: 49.9693 - val_loss: 370.3650 - val_mean_squared_error: 370.3650
Epoch 4/150
43/43 [==============================] - 1s 12ms/step - loss: 46.6268 - mean_squared_error: 46.6268 - val_loss: 363.6172 - val_mean_squared_error: 363.6172
Epoch 5/150
43/43 [==============================] - 1s 12ms/step - loss: 44.8979 - mean_squared_error: 44.8979 - val_loss: 339.5956 - val_mean_squared_error: 339.5956
Epoch 6/150
43/43 [==============================] - 1s 12ms/step - loss: 45.6941 - mean_squared_error: 45.6941 - val_loss: 405.0111 - val_mean_squared_error: 4

In [152]:
mae

[11.444444444444445,
 10.062056737588652,
 9.989316239316238,
 11.26063829787234,
 11.326923076923077,
 12.376068376068377]

In [39]:
mse

[704.1752136752136,
 838.1583924349882,
 1301.52,
 393.6010638297872,
 951.0533333333333,
 326.10470085470087]

In [40]:
r2

[-4.426248174553819,
 -3.6160039762601697,
 -2.2387520383073487,
 0.12676362011866837,
 -2.020044339327833,
 0.11111952392531554]